# Cat and Dog Classification
Classify images of cats and dogs using Histogram of Oriented Gradients (HOG) for feature extraction and Decision Tree for classification. 

In [16]:
import os
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imsave

In [ ]:
# Load dataset
data_dir = './dataset'
images, labels = [], []

for file in os.listdir(data_dir):
    if file.endswith('.jpg'):
        img_path = os.path.join(data_dir, file)
 
        if file.startswith('cat'):
            labels.append(0)  # Cat
        elif file.startswith('dog'):
            labels.append(1)  # Dog
        else:
            continue

        img = imread(img_path)
        img = rgb2gray(img)  # Convert to grayscale
        img_resized = resize(img, (128, 128), anti_aliasing=True)
        images.append(img_resized)

images = np.array(images)
labels = np.array(labels)
print('Images shape:', images.shape)
print('Labels shape:', labels.shape)
print(f'Number of cats: {np.sum(labels == 0)}')
print(f'Number of dogs: {np.sum(labels == 1)}')

In [ ]:
def extract_hog_features(images, visualize=False):
    """Extract HOG features from a list of grayscale images.
    
    Args:
        images: Array of grayscale images (N, 128, 128).
        visualize: If True, return HOG visualizations.
    
    Returns:
        hog_features: Array of HOG features (N, feature_size).
        hog_images: Array of HOG visualizations (N, 128, 128) if visualize=True, else None.
    """
    hog_features = []
    hog_images = [] if visualize else None
  
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    
    for img in images:
        if visualize:
            features, hog_img = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True, feature_vector=True)
            hog_images.append(hog_img)
        else:
            features = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=False, feature_vector=True)
        
        hog_features.append(features)
    
    hog_features = np.array(hog_features)
    if visualize:
        hog_images = np.array(hog_images)
    
    print(f'HOG features shape: {hog_features.shape}')
    return hog_features, hog_images

In [44]:
def split_and_save_dataset(images, labels, test_size=0.2, train_dir='./dog-cat/train', test_dir='./dog-cat/test'):
    """Split dataset into training and test sets and save images to separate directories.
    
    Args:
        images: Array of grayscale images (N, 128, 128).
        labels: Array of labels (0 for cat, 1 for dog).
        test_size: Fraction of data for test set.
        train_dir: Directory to save training images.
        test_dir: Directory to save test images.
    
    Returns:
        X_train, X_test: HOG features for training and test sets.
        y_train, y_test: Labels for training and test sets.
        idx_train, idx_test: Indices for training and test sets.
    """
    # Split the data using stratified sampling to maintain class balance
    indices = np.arange(len(images))
    idx_train, idx_test, y_train, y_test = train_test_split(
        indices, labels, 
        test_size=test_size, 
        random_state=42, 
        stratify=labels
    )
    
    # Extract HOG features for training and test sets
    train_images = images[idx_train]
    test_images = images[idx_test]
    
    X_train, _ = extract_hog_features(train_images, visualize=False)
    X_test, _ = extract_hog_features(test_images, visualize=False)
    
    # Create directories if they don't exist
    os.makedirs(os.path.join(train_dir, 'cats'), exist_ok=True)
    os.makedirs(os.path.join(train_dir, 'dogs'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'cats'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'dogs'), exist_ok=True)
    
    # Save training images
    for i, (idx, label) in enumerate(zip(idx_train, y_train)):
        class_name = 'cats' if label == 0 else 'dogs'
        filename = f'{class_name}_{i}.png'
        filepath = os.path.join(train_dir, class_name, filename)
        # Convert float image to uint8 format for saving
        img_to_save = (images[idx] * 255).astype(np.uint8)
        imsave(filepath, img_to_save)
    
    # Save test images
    for i, (idx, label) in enumerate(zip(idx_test, y_test)):
        class_name = 'cats' if label == 0 else 'dogs'
        filename = f'{class_name}_{i}.png'
        filepath = os.path.join(test_dir, class_name, filename)
        # Convert float image to uint8 format for saving
        img_to_save = (images[idx] * 255).astype(np.uint8)
        imsave(filepath, img_to_save)
    
    print(f'Training set: {len(X_train)} samples')
    print(f'Test set: {len(X_test)} samples')
    print(f'Training cats: {np.sum(y_train == 0)}, dogs: {np.sum(y_train == 1)}')
    print(f'Test cats: {np.sum(y_test == 0)}, dogs: {np.sum(y_test == 1)}')
    
    return X_train, X_test, y_train, y_test, idx_train, idx_test

In [45]:
def train_random_forest(X_train, y_train, n_estimators=100, random_state=42):
    """Train a Random Forest classifier on HOG features.
    
    Args:
        X_train: HOG features for training (N_train, feature_size).
        y_train: Training labels (0 for cat, 1 for dog).
        n_estimators: Number of trees in the forest.
        random_state: Random state for reproducibility.
    
    Returns:
        clf: Trained Random Forest classifier.
    """
    # Initialize Random Forest classifier with optimized parameters
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=random_state,
        n_jobs=-1  # Use all available cores
    )
    
    # Train the classifier
    print("Training Random Forest classifier...")
    clf.fit(X_train, y_train)
    print("Training completed!")
    
    return clf

In [46]:
def evaluate_model(clf, X_test, y_test):
    """Evaluate the Random Forest classifier on the test set.
    
    Args:
        clf: Trained Random Forest classifier.
        X_test: HOG features for test set (N_test, feature_size).
        y_test: Test labels (0 for cat, 1 for dog).
    
    Returns:
        accuracy: Test set accuracy.
        y_pred: Predicted labels for test set.
    """
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy, y_pred

In [ ]:
# Extract HOG features
print("Extracting HOG features...")
hog_features, hog_images = extract_hog_features(images, visualize=True)

# Split and save dataset
print("\nSplitting dataset...")
X_train, X_test, y_train, y_test, idx_train, idx_test = split_and_save_dataset(images, labels, test_size=0.2)

# Train Random Forest
print("\nTraining model...")
clf = train_random_forest(X_train, y_train, n_estimators=150)

# Evaluate model
print("\nEvaluating model...")
accuracy, y_pred = evaluate_model(clf, X_test, y_test)
print(f'\nTest Accuracy: {accuracy:.4f}')

# Visualize results
classes = ['Cat', 'Dog']

# Get sample indices for each class from test set
cat_test_indices = [i for i, idx in enumerate(idx_test) if y_test[i] == 0]
dog_test_indices = [i for i, idx in enumerate(idx_test) if y_test[i] == 1]

sample_indices = list(range(len(idx_test)))

# Create visualization
n_samples = len(sample_indices)
fig, axes = plt.subplots(n_samples, 3, figsize=(15, 4 * n_samples))

if n_samples == 1:
    axes = axes.reshape(1, -1)

for i, test_idx in enumerate(sample_indices):
    # Get the original image index
    original_idx = idx_test[test_idx]
    
    # True and predicted labels
    true_label = y_test[test_idx]
    pred_label = y_pred[test_idx]
    
    # Get prediction probability
    prob = clf.predict_proba(X_test[test_idx:test_idx+1])[0]
    confidence = np.max(prob)
    
    # Plot original image
    axes[i, 0].imshow(images[original_idx], cmap='gray')
    axes[i, 0].set_title(f'Original\nTrue: {classes[true_label]}')
    axes[i, 0].axis('off')
    
    # Plot HOG representation
    axes[i, 1].imshow(hog_images[original_idx], cmap='gray')
    axes[i, 1].set_title('HOG Features')
    axes[i, 1].axis('off')
    
    # Plot prediction result
    color = 'green' if true_label == pred_label else 'red'
    axes[i, 2].text(0.5, 0.6, f'Predicted: {classes[pred_label]}', 
                   ha='center', va='center', fontsize=16, weight='bold')
    axes[i, 2].text(0.5, 0.4, f'Confidence: {confidence:.3f}', 
                   ha='center', va='center', fontsize=14)
    axes[i, 2].text(0.5, 0.2, f'Correct: {"✓" if true_label == pred_label else "✗"}', 
                   ha='center', va='center', fontsize=18, color=color, weight='bold')
    axes[i, 2].set_xlim(0, 1)
    axes[i, 2].set_ylim(0, 1)
    axes[i, 2].set_title('Prediction Result')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

print(f"\nVisualized {len(sample_indices)} samples from the test set")
print(f"Correct predictions: {np.sum([y_test[i] == y_pred[i] for i in sample_indices])}/{len(sample_indices)}")